In [109]:
//import org.json4s._
//import org.json4s.jackson.JsonMethods._
//import org.json4s.DefaultFormats
//implicit val formats = org.json4s.DefaultFormats

import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import org.apache.spark.util.LongAccumulator
import org.apache.log4j._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql._
import org.apache.spark.sql.functions._


import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._
import org.apache.spark.util.LongAccumulator
import org.apache.log4j._
import scala.collection.mutable.ArrayBuffer
import org.apache.spark.sql._
import org.apache.spark.sql.functions._


In [110]:
val rec1: String = """{
    "visitorId": "v1",
    "products": [{
         "id": "i1",
         "interest": 0.68
    }, {
         "id": "i2",
         "interest": 0.42
    }]
}"""

rec1: String =
{
    "visitorId": "v1",
    "products": [{
         "id": "i1",
         "interest": 0.68
    }, {
         "id": "i2",
         "interest": 0.42
    }]
}


In [111]:
val rec2: String = """{
    "visitorId": "v2",
    "products": [{
         "id": "i1",
         "interest": 0.78
    }, {
         "id": "i3",
         "interest": 0.11
    }]
}"""

rec2: String =
{
    "visitorId": "v2",
    "products": [{
         "id": "i1",
         "interest": 0.78
    }, {
         "id": "i3",
         "interest": 0.11
    }]
}


In [112]:
val visitsData: Seq[String] = Seq(rec1, rec2)

visitsData: Seq[String] =
List({
    "visitorId": "v1",
    "products": [{
         "id": "i1",
         "interest": 0.68
    }, {
         "id": "i2",
         "interest": 0.42
    }]
}, {
    "visitorId": "v2",
    "products": [{
         "id": "i1",
         "interest": 0.78
    }, {
         "id": "i3",
         "interest": 0.11
    }]
})


In [113]:
val productIdToNameMap = Map("i1" -> "Nike Shoes", "i2" -> "Umbrella", "i3" -> "Jeans")

productIdToNameMap: scala.collection.immutable.Map[String,String] = Map(i1 -> Nike Shoes, i2 -> Umbrella, i3 -> Jeans)


In [114]:
/*
Now solution starts

*/

In [115]:
import spark.implicits._

import spark.implicits._


In [116]:
import spark.sql

import spark.sql


In [117]:
productIdToNameMap.toSeq.toDF("id","name").createOrReplaceTempView("prodRec")

In [118]:
//sql("select * from prodRec" ).show(false)

In [119]:
for (i<-visitsData)
    {
//      println(rec)
    println("Original Json String is: \n")
    println(i)
    println("\n")
    var rec=spark.read.json(Seq(i).toDS) 
    rec.createOrReplaceTempView("dfVisitsTable")
//    sql("select * from dfVisitsTable").show()
    val productsArr=sql ("SELECT products FROM dfVisitsTable").withColumn("products", explode($"products")).select("products.*")
//    productsArr.show(false)
    productsArr.createOrReplaceTempView("productsArr")

//  Need to do outer join in case the product id in the record is not valid, if product id not found in the MAP,
//  return invalid product
    val enrichedProducts=sql("select a.id, if (b.name is not null, b.name, 'invalid product') name, a.interest from productsArr a full outer join prodRec b on a.id=b.id")
//    enrichedProducts.show(false)
    val enrichedRecord=rec.select("VisitorId").join(enrichedProducts)
//    enrichedRecord.show(false)
    enrichedRecord.createOrReplaceTempView("enrichedRec")
//    sql("select visitorId, collect_list(struct(id, name, interest)) products from enrichedRec group by visitorId").show(false)
    val enrichedJson=sql("select visitorId, collect_list(struct(id, name, interest)) products from enrichedRec group by visitorId").toJSON
    .collect.mkString("",",","")
    println("Enriched Json String is:\n")
    println(enrichedJson)
    println(" ")
    println(" ")
    }

Original Json String is: 

{
    "visitorId": "v1",
    "products": [{
         "id": "i1",
         "interest": 0.68
    }, {
         "id": "i2",
         "interest": 0.42
    }]
}


Enriched Json String is:

{"visitorId":"v1","products":[{"name":"Jeans"},{"id":"i1","name":"Nike Shoes","interest":0.68},{"id":"i2","name":"Umbrella","interest":0.42}]}
 
 
Original Json String is: 

{
    "visitorId": "v2",
    "products": [{
         "id": "i1",
         "interest": 0.78
    }, {
         "id": "i3",
         "interest": 0.11
    }]
}


Enriched Json String is:

{"visitorId":"v2","products":[{"id":"i3","name":"Jeans","interest":0.11},{"id":"i1","name":"Nike Shoes","interest":0.78},{"name":"Umbrella"}]}
 
 
